<a href="https://colab.research.google.com/github/akppan/ml_practice/blob/master/CNN_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk
from nltk import tokenize

In [0]:
import pickle

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,Conv1D,Dropout,MaxPooling1D

Using TensorFlow backend.


In [0]:
from keras import optimizers

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
dataset = pd.read_csv('/content/SMSSpamCollection.csv')

In [0]:
dataset.head()

,Type,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
tags = dataset['Type']
texts = dataset['Message']

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras import metrics

num_max = 1000
le = LabelEncoder()
tags = le.fit_transform(tags)
#print(tags)
# tok = Tokenizer()
tok = Tokenizer(num_words=num_max)
print(tok.fit_on_texts(texts))
# print((tok.fit_on_texts(texts)).shape)
mat_texts = tok.texts_to_matrix(texts,mode='count')
print(mat_texts)

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(texts,tags, test_size = 0.3)
mat_texts_tr = tok.texts_to_matrix(x_train,mode='count')
print(mat_texts_tr)
print(mat_texts_tr.shape)
mat_texts_tst = tok.texts_to_matrix(x_test,mode='count')

max_len = 100
x_train = tok.texts_to_sequences(x_train)
x_test = tok.texts_to_sequences(x_test)
cnn_texts_mat = sequence.pad_sequences(x_train,maxlen=max_len)
max_len = 100
cnn_texts_mat_tst = sequence.pad_sequences(x_test,maxlen=max_len)

None
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 3. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 3. 2. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 2. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 2. 2. ... 0. 0. 0.]]
(3901, 1000)


In [0]:
def get_cnn_model_v1():   
    model = Sequential()
    model.add(Embedding(1000,20,input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,3,padding='valid',activation='relu',strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])
    return model

In [0]:
def check_model(model,xtr,ytr,xts,yts):
    model.fit(xtr,ytr,batch_size=32,epochs=10,verbose=1)
    print(' ')
    model.evaluate(xts,yts)

In [0]:
m1 = get_cnn_model_v1()
check_model(m,cnn_texts_mat,y_train,cnn_texts_mat_tst,y_test)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 20)           20000     
_________________________________________________________________
dropout_17 (Dropout)         (None, 100, 20)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 98, 64)            3904      
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 64)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 256)               16640     
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
activation_11 (Activation)   (None, 256)             

In [0]:
def get_cnn_model_v2():   
    model = Sequential()
    model.add(Embedding(1000,50,input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,3,padding='valid',activation='relu',strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])
    return model

In [0]:
m2 = get_cnn_model_v2()
check_model(m,cnn_texts_mat,y_train,cnn_texts_mat_tst ,y_test)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 100, 50)           50000     
_________________________________________________________________
dropout_19 (Dropout)         (None, 100, 50)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 98, 64)            9664      
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               16640     
_________________________________________________________________
dropout_20 (Dropout)         (None, 256)               0         
_________________________________________________________________
activation_13 (Activation)   (None, 256)             

In [0]:
from keras.layers import LSTM

max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [0]:
def get_cnn_model_v4():    
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])

    print('Train...')
    return model

In [0]:
m3 = get_cnn_model_v4()
check_model(m,cnn_texts_mat,y_train,cnn_texts_mat_tst ,y_test)

Train...
Epoch 1/10
3901/3901 [==============================] - 3s 788us/step - loss: 0.0013 - acc: 0.9997 - binary_accuracy: 0.9997
Epoch 2/10
3901/3901 [==============================] - 2s 601us/step - loss: 0.0014 - acc: 0.9997 - binary_accuracy: 0.9997
Epoch 3/10
3901/3901 [==============================] - 2s 545us/step - loss: 0.0012 - acc: 0.9997 - binary_accuracy: 0.9997
Epoch 4/10
3901/3901 [==============================] - 2s 552us/step - loss: 0.0013 - acc: 0.9997 - binary_accuracy: 0.9997
Epoch 5/10
3901/3901 [==============================] - 2s 552us/step - loss: 0.0011 - acc: 0.9997 - binary_accuracy: 0.9997
Epoch 6/10
3901/3901 [==============================] - 2s 618us/step - loss: 0.0013 - acc: 0.9997 - binary_accuracy: 0.9997
Epoch 7/10
3901/3901 [==============================] - 2s 620us/step - loss: 0.0012 - acc: 0.9997 - binary_accuracy: 0.9997
Epoch 8/10
3901/3901 [==============================] - 2s 627us/step - loss: 0.0011 - acc: 0.9997 - binary_accuracy

In [0]:
def get_simple_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(num_max,)))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])
    return model

In [0]:
m4 = get_simple_model()
check_model(m,mat_texts_tr,y_train,mat_texts_tst,y_test)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 512)               512512    
_________________________________________________________________
dropout_23 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_24 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 257       
Total params: 644,097
Trainable params: 644,097
Non-trainable params: 0
_________________________________________________________________


ValueError: ignored

In [0]:
def get_cnn_model_v3():    
    model = Sequential()
    model.add(Embedding(1000,20,input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(256,3,padding='valid',activation='relu',strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])
    return model

In [0]:
m = get_cnn_model_v3()
check_model(m,cnn_texts_mat,y_train,cnn_texts_mat_tst ,y_test)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 20)           20000     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 20)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 98, 256)           15616     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_3 (Activation)    (None, 256)              

ValueError: ignored

In [0]:
import re
import itertools

In [0]:
def clean_data(message):
    # removing web links
    if(isinstance(message, str)):
        m = [ re.sub(r'http\S+', '', message.lower())]
        ## removing words like goooood and pooooor to good and poor
        m = [''.join(''.join(m)[:2] for _, m in itertools.groupby(m[0]))]
        ## removing appostophes
        m = [remove_appostophes(m[0])]
        ## removing punctuations from the code 
        m = [remove_punctuations(m[0])]
        return m[0]
    else:
        return message

def remove_appostophes(msg):
    if(isinstance(msg, str)):
        APPOSTOPHES = {"s" : "is", "re" : "are", "t": "not", "ll":"will","d":"had","ve":"have","m": "am"}
        words = nltk.tokenize.word_tokenize(msg)
        final_words=[]
        for word in words:
            broken_words=word.split("'")
            for single_words in broken_words:
                final_words.append(single_words)
        reformed = [APPOSTOPHES[word] if word in APPOSTOPHES else word for word in final_words]
        reformed = " ".join(reformed)
        return reformed
    else:
        return msg

def remove_punctuations(msg):
    if(isinstance(msg, str)):
        punctuations = '''!()-[]{};:'"\,./?@#$%^&@*_~'''
        no_punct = ""
        for char in msg:
            if (char not in punctuations):
                no_punct = no_punct + char
        return no_punct
    else:
        return msg



In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [0]:
wnl = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
def my_tokeniser(s):
    if(isinstance(s, str)):
        s = clean_data(s)
        s = s.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    tokens = [t for t in tokens if len(t)>2]
    tokens = [wnl.lemmatize(t) for t in tokens]
    tokens = [t for t in tokens if t not in stop_words]
    return tokens

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
def prepData(text):
    # Convert to array
    textDataArray = my_tokeniser(text)
    print(text)
    txdata = [text]
    print(textDataArray)
    # Convert into list with word ids
    # Features = Tokenizer.texts_to_sequences(text,Tokenizer)
    # Features = keras.preprocessing.text.text_to_word_sequence(text, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
    #Features = keras.preprocessing.text.one_hot(text, 100, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')

    #print(Features)
    # Features = Tokenizer.texts_to_sequences(Tokenizer,textDataArray)
    # Features = pad_sequences(Features, 20, padding='post')
    # tok.fit_on_texts(texts)
    # mat_texts = tok.texts_to_matrix(texts,mode='count')
    mat_texts = tok.texts_to_matrix(txdata,mode='count')
    x_test1 = tok.texts_to_sequences(txdata)
    Features = sequence.pad_sequences(x_test1,maxlen=max_len)

    # Convert into list with word ids
    #Features = Tokenizer.texts_to_sequences(textDataArray,Tokenizer)
    #Features = keras.preprocessing.text.text_to_word_sequence(text, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
    # Features = sequence.pad_sequences(mat_texts, 100, padding='post')
    
    return Features

In [0]:
message = 'Your account has been hacked. Please call 1-800-0000 to speak with a representative'
textTokenizedTest = prepData(message)
predict = (m.predict(textTokenizedTest))

message1 = 'D/P The School will remain closed from 26.10.2019 to 29.10.2019 on account of Diwali and Bhai Dooj. WISHING YOU A HAPPY DIWALI, AUSPICIOUS GOVERDHAN POOJA AND BHAI DOOJ'
textTokenizedTest1 = prepData(message1)
predict1 = (m.predict(textTokenizedTest1))

Your account has been hacked. Please call 1-800-0000 to speak with a representative
['account', 'ha', 'hacked', 'please', 'call', '180000', 'speak', 'representative']
D/P The School will remain closed from 26.10.2019 to 29.10.2019 on account of Diwali and Bhai Dooj. WISHING YOU A HAPPY DIWALI, AUSPICIOUS GOVERDHAN POOJA AND BHAI DOOJ
['school', 'remain', 'closed', '26102019', '29102019', 'account', 'diwali', 'bhai', 'dooj', 'wishing', 'happy', 'diwali', 'auspicious', 'goverdhan', 'pooja', 'bhai', 'dooj']


In [0]:
print(predict)
print(predict1)

[[0.9840577]]
[[1.1923067e-05]]


In [0]:
textTokenizedTest.shape

(1, 100)

In [0]:
message = 'Your account has been hacked. Please call 1-800-0000 to speak with a representative'
textTokenizedTest = prepData(message)
predict2 = (m.predict_classes(textTokenizedTest))

Your account has been hacked. Please call 1-800-0000 to speak with a representative
['account', 'ha', 'hacked', 'please', 'call', '180000', 'speak', 'representative']


In [0]:
predict2

array([[1]], dtype=int32)

In [0]:
message1 = 'D/P The School will remain closed from 26.10.2019 to 29.10.2019 on account of Diwali and Bhai Dooj. WISHING YOU A HAPPY DIWALI, AUSPICIOUS GOVERDHAN POOJA AND BHAI DOOJ'
textTokenizedTest1 = prepData(message1)
predict3 = (m.predict_proba(textTokenizedTest1))

D/P The School will remain closed from 26.10.2019 to 29.10.2019 on account of Diwali and Bhai Dooj. WISHING YOU A HAPPY DIWALI, AUSPICIOUS GOVERDHAN POOJA AND BHAI DOOJ
['school', 'remain', 'closed', '26102019', '29102019', 'account', 'diwali', 'bhai', 'dooj', 'wishing', 'happy', 'diwali', 'auspicious', 'goverdhan', 'pooja', 'bhai', 'dooj']


In [0]:
predict3

array([[1.1923067e-05]], dtype=float32)

In [0]:
predict4 = m.predict_classes(textTokenizedTest1)
predict4

array([[0]], dtype=int32)

In [0]:
pf4 = np.asscalar(predict4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  """Entry point for launching an IPython kernel.


In [0]:
tags_reversed = le.get_params(deep=True)

In [0]:
tags_reversed

{}

In [0]:
def get_integer_mapping(le):
    '''
    Return a dict mapping labels to their integer values
    from an SKlearn LabelEncoder
    le = a fitted SKlearn LabelEncoder
    '''
    res = {}
    for cl in le.classes_:
        res.update({cl:le.transform([cl])[0]})

    return res

In [0]:
dic = get_integer_mapping(le)

In [0]:
dic

{'ham': 0, 'spam': 1}

In [0]:
pf4

0

In [0]:
if(pf4==dic['ham']):
  pfpredi = 'ham'
else:
  pfpredi = 'spam'

In [0]:
pfpredi

'ham'

In [0]:
message1 = 'D/P The School will remain closed from 26.10.2019 to 29.10.2019 on account of Diwali and Bhai Dooj. WISHING YOU A HAPPY DIWALI, AUSPICIOUS GOVERDHAN POOJA AND BHAI DOOJ'
textTokenizedTest1 = prepData(message1)
predict3 = (m.predict_proba(textTokenizedTest1))

In [0]:
message5 = 'Free Hellotune service for 30 days will be activated shortly. Extend validity on Wynk App @wynk.onelink.me/333062766/hello. To stop, call/SMS STOP to 155223'
textTokenizedTest5 = prepData(message5)
predict5 = (m.predict_proba(textTokenizedTest5))
print(predict5)
print(np.asscalar(m.predict(textTokenizedTest5)))

Free Hellotune service for 30 days will be activated shortly. Extend validity on Wynk App @wynk.onelink.me/333062766/hello. To stop, call/SMS STOP to 155223
['free', 'hellotune', 'service', 'day', 'activated', 'shortly', 'extend', 'validity', 'wynk', 'app', 'wynkonelinkme33062766hello', 'stop', 'callsms', 'stop', '155223']
[[0.50586855]]
0.5058685541152954


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  """


In [0]:
message6 = 'CVA \'19Upload the Video clip/PPT of your working prototype latest by 10/11/19 on AICTE-CVA portal.For further details refer your email or AICTE website.'
textTokenizedTest6 = prepData(message6)
predict6 = (m.predict_proba(textTokenizedTest6))
print(predict6)
print(np.asscalar(m.predict(textTokenizedTest6)))

CVA '19Upload the Video clip/PPT of your working prototype latest by 10/11/19 on AICTE-CVA portal.For further details refer your email or AICTE website.
['cva', '19upload', 'video', 'clipppt', 'working', 'prototype', 'latest', '101119', 'aictecva', 'portalfor', 'detail', 'refer', 'email', 'aicte', 'website']
[[0.50293493]]
0.5029349327087402


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  """


In [0]:
message7 = 'Last day to get 50% Off on entire summer collection. Shop online @ Montecarlo.in . Select styles excluded. Easy Refund & Returns'
textTokenizedTest7 = prepData(message7)
predict7 = (m.predict_proba(textTokenizedTest7))
print(predict7)
print(np.asscalar(m.predict(textTokenizedTest7)))
print(np.asscalar(m.predict_classes(textTokenizedTest7)))

Last day to get 50% Off on entire summer collection. Shop online @ Montecarlo.in . Select styles excluded. Easy Refund & Returns
['last', 'day', 'get', 'entire', 'summer', 'collection', 'shop', 'online', 'montecarloin', 'select', 'style', 'excluded', 'easy', 'refund', 'return']
[[0.50494605]]
0.5049460530281067
1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  


In [0]:
message8 = 'Good Morning, Gentle Reminder for  TOI presents VC Conclave @Shangri-La Hotel, Ashoka Road, Janpath, Connaught Place, New Delhi today from 10:00AM onwards.'
textTokenizedTest8 = prepData(message8)
predict8 = (m.predict_proba(textTokenizedTest8))
print(predict8)
print(np.asscalar(m.predict_classes(textTokenizedTest8)))
print(np.asscalar(m.predict(textTokenizedTest8)))

Good Morning, Gentle Reminder for  TOI presents VC Conclave @Shangri-La Hotel, Ashoka Road, Janpath, Connaught Place, New Delhi today from 10:00AM onwards.
['good', 'morning', 'gentle', 'reminder', 'toi', 'present', 'conclave', 'shangrila', 'hotel', 'ashoka', 'road', 'janpath', 'connaught', 'place', 'new', 'delhi', 'today', '1000am', 'onwards']
[[0.5030867]]
1
0.5030866861343384


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  


In [0]:
le1 = LabelEncoder()
tags11 = le1.fit_transform(tags)

In [0]:
get_integer_mapping(le1)

{'ham': 0, 'spam': 1}